In [ ]:
pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 659 kB/s 


In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np

In [ ]:
# db 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [ ]:
# df_main 생성 : maindata
sql = "SELECT * FROM {}".format('MainData')
df_main=pd.read_sql(sql,db_connection)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   위도                497 non-null    float64
 2   경도                497 non-null    float64
 3   생태통로_유형           497 non-null    int64  
 4   차선_수              497 non-null    int64  
 5   등산객_이용빈도          497 non-null    int64  
 6   생태통로_효율성          497 non-null    int64  
 7   이용확인종_수           497 non-null    int64  
 8   폭(m)              270 non-null    float64
 9   연장(m)             271 non-null    float64
 10  높이(m)             21 non-null     float64
 11  개방도               22 non-null     float64
 12  경사도               497 non-null    object 
 13  주변 로드킬 빈도         497 non-null    int64  
 14  산책로까지의 최단 거리(km)  497 non-null    float64
 15  농가까지의 거리(km)      497 non-null    float64
 16  도로 최고제한속도(km/h)   497 non-null    int64  
 1

In [ ]:
# df_size 생성 : 생태통로_규격_실측 data
sql = "SELECT * FROM {}".format('생태통로_규격_실측')
df_size=pd.read_sql(sql,db_connection)
df_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  306 non-null    int64  
 1   번호          306 non-null    int64  
 2   위도_십진법      306 non-null    float64
 3   경도_십진법      306 non-null    float64
 4   생태통로타입      306 non-null    object 
 5   면적_m^2      2 non-null      float64
 6   너비_m        240 non-null    float64
 7   길이_m        240 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 19.2+ KB


In [ ]:
# '번호' 열을 기준으로 df_main과 df_size 병합
coord = pd.merge(df_main[['번호', '위도', '경도']], df_size[['번호', '위도_십진법', '경도_십진법']], how='left', on='번호')

# df_size의 위도, 경도가 null이 아닌 인덱스 idx에 저장
idx = list(coord[((coord['위도_십진법'].notnull()) & coord['경도_십진법'].notnull())].index)

# df_main의 위도, 경도를 df_size의 위도, 경도로 변경
for i in idx:
  df_main.loc[i,'위도']=coord.loc[i,'위도_십진법']
  df_main.loc[i,'경도']=coord.loc[i,'경도_십진법']

# df_main[df_main['번호']==1250]

,번호,위도,경도,생태통로_유형,차선_수,등산객_이용빈도,생태통로_효율성,이용확인종_수,폭(m),연장(m),...,식물군락명,등산로까지 최단거리(km),유도울타리_위도_2.3km,유도울타리_경도_2.3km,유도울타리_연장_m,유도울타리_높이_m,교통량,환경영향평가점수,건물까지거리(km),최우점식물군락
89,1250,37.75362,127.42239,0,1,0,3,0,444.0,16.0,...,잣나무식재림,0.204366,0.0,0.0,0,0.0,12042.875,2,0.054647,잣나무


In [ ]:
# df → db
# table_name = 'MainData'
# df_main.to_sql(name=table_name, con=db_connection, if_exists='replace', index=False)